In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import header
paths = header.setup_environment()
import platform
if platform.system()=='Darwin':
    INPUTDIR = '/Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA/WellData/MERGED'
else:
    INPUTDIR = os.path.join(paths['new_data'], '03_merge_inventories')
import libWellData as LLE
transducersDF = LLE.get_transducers_dataframe(paths)
display(transducersDF)


In [ ]:
csvfile = 'dailyshifts.csv'
aircolumns = ['AirPressureShallow', 'AirPressureDeep', '1226420', '1226429']
watercolumns = ['1226419', '1226421', '2151691', '2149882', '1226423', '2151692']

if os.path.isfile(csvfile):
    dfshifts = pd.read_csv('dailyshifts.csv', index_col=None)
    dfshifts['datetime'] = pd.to_datetime(dfshifts['datetime'])
    dfshifts['1226420'] *= -1 + dfshifts['AirPressureShallow']
    dfshifts['1226429'] *= -1 + dfshifts['AirPressureShallow']
    dfshifts.plot(x='datetime', y=aircolumns[2:], ylabel='DC shift (PSI) to align with analog barometers')
    dfshifts.plot(x='datetime', y=watercolumns, ylabel='estimated set depth (m)')

print(f'Mean DC shift for digital barometers:')
for db in aircolumns[2:]:
    this_transducer = transducersDF[transducersDF['serial']==db]
    print(f'{db}: calculated={dfshifts[db].mean():.5f} PSI, bp={this_transducer.iloc[0]["bp"]} PSI')

print('\nMeasured set depths:')
for db in watercolumns:
    this_transducer = transducersDF[transducersDF['serial']==db]
    estimated = dfshifts[db].mean()/0.3048
    measured = float(this_transducer["set_depth_ft"])
    print(f'{db}: measured={measured:9.5f}, estimated={estimated:9.5f} feet, difference={estimated-measured:9.5f} feet')
